In [5]:
from controllables.energyplus import (
    World,
    #WeatherModel,
    #Report,
    Actuator,
    OutputVariable,
)

from energyplus.dataset.basic import dataset as _epds_

world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=World.Specs.Output(
        report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    runtime=World.Specs.Runtime(
        #recurring=True,
        #design_day=True,
    ),
)

In [6]:
from controllables.core.tools.gymnasium import (
    SpaceVariableContainer,
    VariableBox,
)

import gymnasium as _gymnasium_
import numpy as _numpy_


env = SpaceVariableContainer(
    action_space=_gymnasium_.spaces.Dict({
        'thermostat': VariableBox(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='CORE_MID',
        ))
    }),    
    observation_space=_gymnasium_.spaces.Dict({
        'temperature': VariableBox(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(OutputVariable.Ref(
            type='People Air Temperature',
            key='CORE_MID',
        )),
    }),
).__attach__(world)

@world.on('end_zone_timestep_after_zone_reporting')
def _(_):
    env.observation.value
    env.observe()

    # env.action.value = {
    #     'thermostat': 20,
    # }
    # env.act(...)

# world.on('end_zone_timestep_after_zone_reporting', ...)

In [7]:
world.awaitable.run()

<Task pending name='Task-7' coro=<World.run() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/core/utils/awaitables.py:81>>

In [8]:

for _ in range(10):
    ctx = await world.events['end_zone_timestep_after_zone_reporting'].awaitable(deferred=True)
    print(world['wallclock'].value, env.observe())
    env.act({'thermostat': 20})
    ctx.ack()

2024-12-21 00:14:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 00:29:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 00:44:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 00:59:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 01:14:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 01:29:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 01:44:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 01:59:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 02:14:00+00:00 {'temperature': array(0., dtype=float32)}
2024-12-21 02:29:00+00:00 {'temperature': array(0., dtype=float32)}


In [9]:
#event_ctx, = ctx.input
#event_ctx.event